<a href="https://colab.research.google.com/github/profteachkids/subtitle_generator/blob/main/subtitle_generate_color_animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Generate Text

In [ ]:
!wget https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf

--2021-10-03 14:32:20--  https://github.com/profteachkids/subtitle_generator/raw/main/Dosis-Bold.ttf
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/Dosis-Bold.ttf [following]
--2021-10-03 14:32:20--  https://raw.githubusercontent.com/profteachkids/subtitle_generator/main/Dosis-Bold.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92920 (91K) [application/octet-stream]
Saving to: ‘Dosis-Bold.ttf.1’

Dosis-Bold.ttf.1    100%[===================>]  90.74K  --.-KB/s    in 0.01s   

2021-10-03 14:32:20 (6.43 MB/s) - ‘Dosis-Bold.ttf.1’ saved [92920/92920]



In [54]:
import os
import numpy as np
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps
import requests
from io import BytesIO
from google.colab import files
import shutil
from datetime import date

colors={'w':(255, 255, 255, 255),
    'k':(0, 0, 0, 255),   #black
    'r':(255, 0, 0, 255),    #red
    '1':(255, 0, 221, 255),  #pink
    'b':(0,0,255,255), #blue
    '2':(0,255,69,255),
    '3':(128,0,128,255), #purple
    'o':(255,165,0,255), #orange
    'y':(255,255,0,255)} #yellow

text="""after i was betrayed
by my minecraft friend
and surrounded by my
enemies on the
public smp
in the last part
i was done being
pushed around and
decided to muster up
my guts and %rkk%fight%rkk% back
now unlike what
happened in a previous
video this time didn't go
as well...
my armor was %okk%breaking
i was running %okk%low%okk% on
materials so i did what
anyone in my position
would do
hide my gear and run
that plan would've been
so much better if i still
didn't %rkk%die 
but its safe to say that
my enemies still aren't
the top 10 happiest people
on the public smp right now
help me in this war
server ip in Discord""" 

rootname='subtitles'+date.today().strftime('%d%b%y')

root='./'+rootname
try:
    shutil.rmtree(rootname)
except:
    pass
os.makedirs(root)

imw,imh=1920,250

fnt = ImageFont.truetype("Dosis-Bold.ttf", 115)

In [45]:
text_lines=text.split('\n')
for i,text_line in enumerate(text_lines):
    z = re.match(r"%\w{3}%", text_line)
    if not z:
        text_lines[i]='%wkk%'+text_line
    
i=0
w_start=0
filenames=[]
w_starts, w_ends=[],[]
for text_line in text_lines:
    z = re.split(r"(%\w{3}%)", text_line)
    color_strings = z[1::2]
    text_strings = z[2::2]

    full_text_line = ''.join(text_strings)
    w_end = w_start + len(full_text_line.split())
    w_starts.append(w_start)
    w_ends.append(w_end)

    filename = f'a_{i:04d}_{w_start:04d}_{w_end:04d}.png'
    fullname=root+ '/' + filename
    i+=1
    w_start = w_end

    im = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d = ImageDraw.Draw(im)
    imface = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d2 = ImageDraw.Draw(imface)

    w,h=d.textsize(full_text_line,fnt)
    x,y = (imw-w)//2, (imh-h)//2
    prior_text=''
    for color_string, text_string in zip(color_strings,text_strings):
        font_c = color_string[1]
        outline_c = color_string[2]
        shadow_c = color_string[3]

        d.text((x,y), text_string, font=fnt, fill=colors[shadow_c])
        d2.text((x,y), text_string, font=fnt, fill=colors[font_c], stroke_width=7, stroke_fill=colors[outline_c])

        prior_text+=text_string
        x+=d.textsize(text_string,fnt)[0]

    im = im.filter(ImageFilter.GaussianBlur(radius=30))
    imnp_raw=np.asarray(im)
    imnp=imnp_raw.copy()
    imnp[:,:,3]=(imnp_raw[:,:,3]*(255.0/imnp_raw[:,:,3].max())).astype(np.uint8)
    imfacenp = np.asarray(imface)
    blank = np.zeros((imh,imw,4),dtype=np.uint8)
    compnp=np.where((imnp[:,:,3]>0)[:,:,None], imnp, blank)
    compnp=np.where((imfacenp[:,:,3]>200)[:,:,None], imfacenp, compnp)

    Image.fromarray(compnp,'RGBA').save(fullname)
    filenames.append(filename)
shutil.make_archive(rootname,'zip',root_dir =root)

'/content/subtitles03Oct21.zip'

In [47]:

timeline_end="""                    <enabled>TRUE</enabled>
                    <locked>FALSE</locked>
                </track>
                <format>
                    <samplecharacteristics>
                        <width>1920</width>
                        <height>1080</height>
                        <pixelaspectratio>square</pixelaspectratio>
                        <rate>
                            <timebase>60</timebase>
                            <ntsc>FALSE</ntsc>
                        </rate>
                        <codec>
                            <appspecificdata>
                                <appname>Final Cut Pro</appname>
                                <appmanufacturer>Apple Inc.</appmanufacturer>
                                <data>
                                    <qtcodec/>
                                </data>
                            </appspecificdata>
                        </codec>
                    </samplecharacteristics>
                </format>
            </video>
            <audio>
                <track>
                    <enabled>TRUE</enabled>
                    <locked>FALSE</locked>
                </track>
            </audio>
        </media>
    </sequence>
</xmeml>
"""

clipitem_mid="""
        <timecode>
            <string>00:00:00:00</string>
            <displayformat>NDF</displayformat>
            <rate>
                <timebase>60</timebase>
                <ntsc>FALSE</ntsc>
            </rate>
        </timecode>
        <media>
            <video>
                <duration>1</duration>
                <samplecharacteristics>
                    <width>1920</width>
                    <height>1080</height>
                </samplecharacteristics>
            </video>
        </media>
    </file>
    <compositemode>normal</compositemode>
    <filter>
        <enabled>TRUE</enabled>
        <start>0</start>
        <end>3600001</end>
        <effect>
            <name>Basic Motion</name>
            <effectid>basic</effectid>
            <effecttype>motion</effecttype>
            <mediatype>video</mediatype>
            <effectcategory>motion</effectcategory>
            <parameter>
                <name>Scale</name>
                <parameterid>scale</parameterid>
                <value>100</value>
                <keyframe>
                    <when>216000</when>
                    <value>85</value>
                </keyframe>
                <keyframe>
                    <when>216015</when>
                    <value>100</value>
                    <inbez>
                        <horiz>-3</horiz>
                        <vert>0</vert>
                    </inbez>
                </keyframe>
                <valuemin>0</valuemin>
                <valuemax>10000</valuemax>
            </parameter>
            <parameter>
                <name>Center</name>
                <parameterid>center</parameterid>
                <value>
                    <horiz>0</horiz>
                    <vert>0</vert>
                </value>
            </parameter>
            <parameter>
                <name>Rotation</name>
                <parameterid>rotation</parameterid>
                <value>0</value>
                <valuemin>-100000</valuemin>
                <valuemax>100000</valuemax>
            </parameter>
            <parameter>
                <name>Anchor Point</name>
                <parameterid>centerOffset</parameterid>
                <value>
                    <horiz>0</horiz>
                    <vert>0</vert>
                </value>
            </parameter>
        </effect>
    </filter>
    <filter>
        <enabled>TRUE</enabled>
        <start>0</start>
        <end>3600001</end>
        <effect>
            <name>Crop</name>
            <effectid>crop</effectid>
            <effecttype>motion</effecttype>
            <mediatype>video</mediatype>
            <effectcategory>motion</effectcategory>
            <parameter>
                <name>left</name>
                <parameterid>left</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
            <parameter>
                <name>right</name>
                <parameterid>right</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
            <parameter>
                <name>top</name>
                <parameterid>top</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
            <parameter>
                <name>bottom</name>
                <parameterid>bottom</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>100</valuemax>
            </parameter>
        </effect>
    </filter>"""


clipitem_end="""    <filter>
        <enabled>TRUE</enabled>
        <start>-1</start>
        <end>-1</end>
        <effect>
            <name>Time Remap</name>
            <effectid>timeremap</effectid>
            <effecttype>motion</effecttype>
            <mediatype>video</mediatype>
            <effectcategory>motion</effectcategory>
            <parameter>
                <name>speed</name>
                <parameterid>speed</parameterid>
                <value>0</value>
                <valuemin>-10000</valuemin>
                <valuemax>10000</valuemax>
            </parameter>
            <parameter>
                <name>reverse</name>
                <parameterid>reverse</parameterid>
                <value>FALSE</value>
            </parameter>
            <parameter>
                <name>frameblending</name>
                <parameterid>frameblending</parameterid>
                <value>FALSE</value>
            </parameter>
            <parameter>
                <name>variablespeed</name>
                <parameterid>variablespeed</parameterid>
                <value>0</value>
                <valuemin>0</valuemin>
                <valuemax>1</valuemax>
            </parameter>
            <parameter>
                <name>graphdict</name>
                <parameterid>graphdict</parameterid>
                <keyframe>
                    <when>0</when>
                    <value>0</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfstart>TRUE</speedkfstart>
                </keyframe>
                <keyframe>
                    <when>216000</when>
                    <value>0</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfin>TRUE</speedkfin>
                </keyframe>
                <keyframe>
                    <when>216013</when>
                    <value>0</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfout>TRUE</speedkfout>
                </keyframe>
                <keyframe>
                    <when>3600001</when>
                    <value>1</value>
                    <speedvirtualkf>TRUE</speedvirtualkf>
                    <speedkfend>TRUE</speedkfend>
                </keyframe>
                <valuemin>0</valuemin>
                <valuemax>0</valuemax>
                <interpolation>
                    <name>FCPCurve</name>
                </interpolation>
            </parameter>
        </effect>
    </filter>
</clipitem>
"""

In [48]:
timeline=timeline_start
for i,filename in enumerate(filenames):
    start = 30*i
    end = start+30
    clipitem_start = f"""<clipitem id="{filename} 0">
    <name>{filename}</name>
    <duration>3600001</duration>
    <rate>
        <timebase>60</timebase>
        <ntsc>FALSE</ntsc>
    </rate>
    <start>{start}</start>
    <end>{end}</end>
    <enabled>TRUE</enabled>
    <in>{216000}</in>
    <out>{216000+end-start}</out>
    <file id="{filename} 2">
        <duration>1</duration>
        <rate>
            <timebase>60</timebase>
            <ntsc>FALSE</ntsc>
        </rate>
        <name>{filename}</name>
        <pathurl>file://localhost/{path}/{filename}</pathurl>"""   
    clipitem_opacity=f"""
                        <filter>
                            <enabled>TRUE</enabled>
                            <start>0</start>
                            <end>3600001</end>
                            <effect>
                                <name>Opacity</name>
                                <effectid>opacity</effectid>
                                <effecttype>motion</effecttype>
                                <mediatype>video</mediatype>
                                <effectcategory>motion</effectcategory>
                                <parameter>
                                    <name>opacity</name>
                                    <parameterid>opacity</parameterid>
                                    <value>100</value>
                                    <keyframe>
                                        <when>{216000+end-1}</when>
                                        <value>100</value>
                                    </keyframe>
                                    <keyframe>
                                        <when>{216000+end}</when>
                                        <value>0</value>
                                    </keyframe>
                                    <valuemin>0</valuemin>
                                    <valuemax>100</valuemax>
                                </parameter>
                            </effect>
                        </filter>""" 
    timeline+=clipitem_start + clipitem_mid + clipitem_opacity+ clipitem_end
timeline+=timeline_end

##Imports and Silerio

In [ ]:
!pip install biopython
!pip install bidict
!pip install torchaudio
!wget https://github.com/profteachkids/subtitle_generator/raw/main/en_v5.jit

     |████████████████████████████████| 2.3 MB 5.0 MB/s 
     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 831.4 MB 5.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
--2021-10-03 14:38:22--  https://github.com/profteachkids/subtitle_generator/raw/main/en_v5.jit
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubuserco

In [ ]:
import subprocess as sp
import os
import torch
import pandas as pd
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from bidict import bidict
import string
from itertools import groupby

class Decoder():
    def __init__(self,labels):
        self.labels = labels
        self.blank_idx = self.labels.index('_')
        self.space_idx = self.labels.index(' ')

    def process(self,
                probs, wav_len, word_align):
        assert len(self.labels) == probs.shape[1]
        for_string = []
        argm = torch.argmax(probs, axis=1)
        align_list = [[]]
        for j, i in enumerate(argm):
            if i == self.labels.index('2'):
                try:
                    prev = for_string[-1]
                    for_string.append('$')
                    for_string.append(prev)
                    align_list[-1].append(j)
                    continue
                except:
                    for_string.append(' ')
                    warnings.warn('Token "2" detected a the beginning of sentence, omitting')
                    align_list.append([])
                    continue
            if i != self.blank_idx:
                for_string.append(self.labels[i])
                if i == self.space_idx:
                    align_list.append([])
                else:
                    align_list[-1].append(j)

        string = ''.join([x[0] for x in groupby(for_string)]).replace('$', '').strip()

        align_list = list(filter(lambda x: x, align_list))

        if align_list and wav_len and word_align:
            align_dicts = []
            linear_align_coeff = wav_len / len(argm)
            to_move = min(align_list[0][0], 1.5)
            for i, align_word in enumerate(align_list):
                if len(align_word) == 1:
                    align_word.append(align_word[0])
                align_word[0] = align_word[0] - to_move
                if i == (len(align_list) - 1):
                    to_move = min(1.5, len(argm) - i)
                    align_word[-1] = align_word[-1] + to_move
                else:
                    to_move = min(1.5, (align_list[i+1][0] - align_word[-1]) / 2)
                    align_word[-1] = align_word[-1] + to_move

            for word, timing in zip(string.split(), align_list):
                align_dicts.append({'word': word,
                                    'start_ts': round(timing[0] * linear_align_coeff, 2),
                                    'end_ts': round(timing[-1] * linear_align_coeff, 2)})

            return string, align_dicts
        return string

    def __call__(self,
                 probs: torch.Tensor,
                 wav_len: float = 0,
                 word_align: bool = False):
        return self.process(probs, wav_len, word_align)

torch.set_grad_enabled(False)

model = torch.jit.load("en_v5.jit", map_location=torch.device('cpu'))
model.eval()
decoder=Decoder(model.labels)

DEVNULL = open(os.devnull, "w")

def ffmpeg_load_audio(
    filename,
    sr=44100,
    mono=False,
    normalize=True,
    in_type=np.int16,
    out_type=np.float32,
):
    channels = 1 if mono else 2
    format_strings = {
        np.float64: "f64le",
        np.float32: "f32le",
        np.int16: "s16le",
        np.int32: "s32le",
        np.uint32: "u32le",
    }
    format_string = format_strings[in_type]
    command = [
        "ffmpeg",
        "-i",
        filename,
        "-f",
        format_string,
        "-acodec",
        "pcm_" + format_string,
        "-ar",
        str(sr),
        "-ac",
        str(channels),
        "-",
    ]
    p = sp.Popen(command, stdout=sp.PIPE, stderr=DEVNULL, bufsize=4096)
    bytes_per_sample = np.dtype(in_type).itemsize
    frame_size = bytes_per_sample * channels
    chunk_size = frame_size * sr  # read in 1-second chunks
    raw = b""
    with p.stdout as stdout:
        while True:
            data = stdout.read(chunk_size)
            if data:
                raw += data
            else:
                break
    audio = np.fromstring(raw, dtype=in_type).astype(out_type)
    if channels > 1:
        audio = audio.reshape((-1, channels)).transpose()
    if audio.size == 0:
        return audio, sr
    if issubclass(out_type, np.floating):
        if normalize:
            peak = np.abs(audio).max()
            if peak > 0:
                audio /= peak
        elif issubclass(in_type, np.integer):
            audio /= np.iinfo(in_type).max
    return audio, sr

## Timeline

In [61]:


def to_text(f):
    print(f)
    audio, sr = ffmpeg_load_audio(
        f, sr=16000, in_type=np.float32, out_type=np.float32
    )
    audio = np.expand_dims(audio.astype(np.float32)[0], 0)

    res = []
    batch_size =  3*sr
    N = audio.size // batch_size

    sections = np.array_split(audio, N, 1)
    section_start=0
    for i, section in enumerate(sections):
        input=torch.from_numpy(section)
        output = model(input)[0]
        decoded=decoder(output.cpu(), section.size, word_align=True)
        if len(decoded)==2:
            s,dlist = decoded
            for d in dlist:
                d["start_ts"] = (d["start_ts"] + section_start)/sr
                d["end_ts"] = (d["end_ts"] +  section_start)/sr
                print(d["word"], d["start_ts"], d["end_ts"])
                res.append(d)
            section_start+=section.size


    df = pd.DataFrame(res)
    df.to_csv("voice_times.csv")

    voice_words = df["word"].values
    print(" ".join(voice_words))

def align(rawtext):
    df = pd.read_csv("voice_times.csv")
    voice_words = df["word"].values
    start_times = df["start_ts"].values
    end_times = df["end_ts"].values

    all_words = set(rawtext).union(set(voice_words))

    word_dict = bidict(zip(all_words, range(len(all_words))))
    word_dict["-"] = "-"
    voice = [word_dict[word] for word in voice_words]
    script = [word_dict[word] for word in rawtext]
    align = pairwise2.align.globalxx(voice, script, gap_char =['-'], one_alignment_only=True)[0]
    matches=[]
    mismatches=[]
    mismatch_voice=word_dict.inverse[align.seqA[0]]   
    mismatch_text=word_dict.inverse[align.seqB[0]]
    v_end_pos=0
    v_start_pos=0
    t_end_pos=0
    t_start_pos=0
    for i,(v, t) in enumerate(zip(align.seqA[1:], align.seqB[1:])):
        print(word_dict.inverse[v], word_dict.inverse[t])
        if v==t:
            mismatches.append([t_start_pos, t_end_pos, start_times[v_start_pos], end_times[v_end_pos], mismatch_voice, mismatch_text])
            v_end_pos+=1
            v_start_pos=v_end_pos
            t_end_pos+=1
            t_start_pos=t_end_pos
            mismatch_voice=word_dict.inverse[v]
            mismatch_text=word_dict.inverse[t]
        else:
            if word_dict.inverse[v] != '-' :
                v_end_pos+=1
            if word_dict.inverse[t] != '-' :
                t_end_pos+=1
            mismatch_voice+=' ' + word_dict.inverse[v]
            mismatch_text+=' ' + word_dict.inverse[t]


    mismatches.append([t_start_pos, t_end_pos, start_times[v_start_pos], end_times[v_end_pos], mismatch_voice, mismatch_text])

    idx=[]
    times=[]
    for t_start_pos, t_end_pos, start_time, end_time, v, t in mismatches:
        print('-'*30)
        print(v)
        print(t)
        print(t_start_pos,t_end_pos, start_time, end_time)
        d={}

        t2 = t.translate(t.maketrans(string.punctuation, " " * len(string.punctuation)))
        t2 = " ".join(t2.split())

        idx.append(t_start_pos)
        times.append(start_time)

    idx.append(t_end_pos+1)
    times.append(end_time)

    return np.array(idx),np.array(times)

In [78]:

def create_timeline(idx, times):

    timeline=timeline_start
    for filename,w_start,w_end in zip(filenames,w_starts,w_ends):
        start, end = (np.interp([w_start, w_end], idx, times)*60).astype(np.int64)

        clipitem_start = f"""<clipitem id="{filename} 0">
        <name>{filename}</name>
        <duration>3600001</duration>
        <rate>
            <timebase>60</timebase>
            <ntsc>FALSE</ntsc>
        </rate>
        <start>{start}</start>
        <end>{end}</end>
        <enabled>TRUE</enabled>
        <in>{216000}</in>
        <out>{216000+end-start}</out>
        <file id="{filename} 2">
            <duration>1</duration>
            <rate>
                <timebase>60</timebase>
                <ntsc>FALSE</ntsc>
            </rate>
            <name>{filename}</name>
            <pathurl>file://localhost/{path}/{filename}</pathurl>"""   
        clipitem_opacity=f"""
                            <filter>
                                <enabled>TRUE</enabled>
                                <start>0</start>
                                <end>3600001</end>
                                <effect>
                                    <name>Opacity</name>
                                    <effectid>opacity</effectid>
                                    <effecttype>motion</effecttype>
                                    <mediatype>video</mediatype>
                                    <effectcategory>motion</effectcategory>
                                    <parameter>
                                        <name>opacity</name>
                                        <parameterid>opacity</parameterid>
                                        <value>100</value>
                                        <keyframe>
                                            <when>{216000+end-1}</when>
                                            <value>100</value>
                                        </keyframe>
                                        <keyframe>
                                            <when>{216000+end}</when>
                                            <value>0</value>
                                        </keyframe>
                                        <valuemin>0</valuemin>
                                        <valuemax>100</valuemax>
                                    </parameter>
                                </effect>
                            </filter>""" 
        timeline+=clipitem_start + clipitem_mid + clipitem_opacity+ clipitem_end
    timeline+=timeline_end
    with open('Timeline'+date.today().strftime('%d%b%y')+'.xml','w') as f:
        f.write(timeline)
    


In [79]:

path='D:/Downloads/'+rootname
rawtext = re.sub(r"%\w{3}%",'',text)
rawtext = re.sub(r"[,.!?-] ", " ", rawtext, 0, re.MULTILINE)
rawtext = rawtext.lower()
rawtext = rawtext.split()
to_text("audio.mov")
idx, times = align(rawtext)
duration = int(times[-1]*60)
timeline_start=f"""<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE xmeml>
<xmeml version="5">
    <sequence>
        <name>Timeline 1 (Resolve)</name>
        <duration>{duration}</duration>
        <rate>
            <timebase>60</timebase>
            <ntsc>FALSE</ntsc>
        </rate>
        <in>-1</in>
        <out>-1</out>
        <timecode>
            <string>01:00:00:00</string>
            <frame>216000</frame>
            <displayformat>NDF</displayformat>
            <rate>
                <timebase>60</timebase>
                <ntsc>FALSE</ntsc>
            </rate>
        </timecode>
        <media>
            <video>
                <track>
"""   
create_timeline(idx, times)

audio.mov


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


after 0.0 0.197925625
betred 0.5146068749999999 1.0292137499999998
by 1.0292137499999998 1.1875550000000001
my 1.1875550000000001 1.345895
micrro 1.345895 1.781331875
friendon 1.781331875 2.098013125
in 2.098013125 2.29593875
suranvam 2.29593875 2.8897162499999998
means 3.1668125 3.48349375
on 3.48349375 3.6418343749999997
the 3.6418343749999997 3.8001750000000003
public 3.8001750000000003 3.958515625
hasn'amin 3.958515625 4.552293125
last 4.71063375 4.908559374999999
part 4.908559374999999 5.185655625000001
i 5.185655625000001 5.38358125
was 5.38358125 5.660677499999999
done 5.660677499999999 5.898188125000001
be 5.97735875 6.214869375
str 6.333625 6.5315506249999995
and 6.610721249999999 6.808646875
decided 6.808646875 7.204498125
the 7.204498125 7.402424375000001
of 7.560765 7.758690625
my 7.758690625 7.91703125
guts 7.91703125 8.2337125
and 8.2337125 8.392053125
fight 8.392053125 8.708734375
back 8.708734375 8.90666
now 8.985830625 9.223341249999999
like 9.5004375 9.737948125
what 